# master Git revision: l1_anomaly_ae/dnn/end2end.py (EXPLANATION)

We have all the code below. Let's break it down the code step by step to understand how it works in detail. Basically, this script is designed to train and evaluate DNN models with CMS L1 emulator inputs, leveraging TensorFlow, data handling, and plotting.

In [ ]:
import argparse
import os
from typing import Any, Dict, List, Optional, Tuple, Union

import numpy as np
import h5py as h5

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
from tensorflow import keras

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

from matplotlib import pyplot as plt


from utilities.configuration_classes import Config
from utilities.utilities import set_level, set_mpl_style, patch_mpl, set_global_rand_seed, dump_full_h5, load_full_h5
from utilities.utilities import info, warn, error, debug
from model.losses import configure_loss

from workflow.hls import hls

from workflow.plot_manager import HistPlotManager, RoCPlotManager


from workflow.train import train
from workflow.dataloader import DataLoader
from workflow.prepare_evaluation_object import prepare_evaluation_object
from workflow.trim_encoder import get_trimmed_vae, generate_trimmed_vae
from workflow.legacy_plot import legacy_plot


def get_bits(s: str, pos: int, length: int = 4):
    if len(s) <= 2 and s.isdigit():
        i = int(s)
        assert 0 <= i <= 15
        s = bin(i)[2:].zfill(4)[::-1]
    if len(s) != length:
        return False
    for c in s:
        if c != '0' and c != '1':
            return False
    return s[pos] == '1'


if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Train and test model')
    parser.add_argument('--config', type=str, default='dnn/config.yml', help='Configuration file with IO and training parameters setup')
    parser.add_argument('--run', type=str, default='all', help='what to run: train, eval, or all')
    parser.add_argument('--verbose-level', '-v', type=str, help='verbosity level, number or name', default='INFO')
    parser.add_argument('--force-cache-refresh', '-fr', action='store_true', help='force refresh of cache')
    args = parser.parse_args()
    set_level(args.verbose_level)

    info(f"Using config file from {args.config}")
    config = Config.from_file(args.config)

    if os.path.isdir(config.output_path):
        if config.silent_overwrite:
            warn(f"Output path {config.output_path} already exists. Will overwriting silently.")
        else:
            input("Warning: output directory exists. Press Enter to continue...")
    else:
        os.mkdir(config.output_path)

    debug(f"Save config file to {config.output_path + '/config.json'}")
    config.dump(config.output_path + '/config.json')

    dataloader = DataLoader(config.data)

    set_mpl_style()
    if config.save_json_for_plots:
        patch_mpl()

    if config.global_seed is None:
        config.global_seed = np.random.randint(0, 1000000)

    info(f"Global random seed set to {config.global_seed}")
    set_global_rand_seed(config.global_seed)

    if args.force_cache_refresh:
        dataloader.load(force_refresh=True)

    if config.train.deterministic:
        tf.config.experimental.enable_op_determinism()
        warn("Deterministic mode is enabled. Performance may degrade.")

    if args.run == 'all' or 'train' in args.run or get_bits(args.run, 0):
        dataset = dataloader.get_dataset()
        scales, biases = dataset.norm_scale, dataset.norm_bias
        with h5.File(config.output_path + '/scales.h5', 'w') as f:
            f.create_dataset('norm_scale', data=scales)
            f.create_dataset('norm_bias', data=biases)
            
            
        configure_loss(dataset.norm_scale, dataset.norm_bias, config.data.constituents_mask)
        train(config, dataloader.get_dataset())
        generate_trimmed_vae(config)        

    if args.run == 'all' or 'eval' in args.run or get_bits(args.run, 1):
        dataset = dataloader.get_dataset()
        configure_loss(dataset.norm_scale, dataset.norm_bias, config.data.constituents_mask)
        prepare_evaluation_object(config, dataset)
        hls_model = hls(config, do_csim=True, dataset=dataset)

    if args.run == 'all' or 'plot' in args.run or get_bits(args.run, 2):
        results = load_full_h5(config.output_path + '/results.h5')
        hist_plot_manager = HistPlotManager(config, results)
        roc_plot_manager = RoCPlotManager(config, results)
        input_plot_manager = InputPlotManager(config, results)
        roc_plot_manager.plot_eff_table()
        hist_plot_manager.plt_hls_dist(config.hls_metric)
        roc_plot_manager.plot_all_roc_curves(16)
        input_plot_manager.plot_all_input_hist()
        input_plot_manager.plot_all_input_correlation()

    if args.run == 'all' or 'hls' in args.run or get_bits(args.run, 3):
        hls_model = hls(config, do_csim=False, do_synth=True)

Let's see each part of the code:
### **Import Statements**

In [ ]:
import argparse
import os
from typing import Any, Dict, List, Optional, Tuple, Union

import numpy as np
import h5py as h5

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
from tensorflow import keras

* `argparse`: Handles command-line arguments. This allows the script to be more flexible and cofigurable without the need to modify the code.
* `os`: Provides a way to use operating system dependent functionality. We can interact with the Operative System.
* `typing`: Provides type hints.
* `numpy` (np): Numerical operations (numerical arrays).
* `h5py` (h5): Interacts with HDF5 files. It imports HDF5 to handle data storage files in HDF5 format.
* `os.environ['TF_CPP_MIN_LOG_LEVEL']`: Sets TensorFlow logging level to minimize unnecessary logs and shows only eror messages (level 3).
* `tensorflow` (tf): Core library for building and training models.
* `tensorflow.keras`: High-level API for building neural networks.

### **GPU Configuration**

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

* `tf.config.experimental.list_physical_devices('GPU')`: Lists available GPUs.
* `tf.config.experimental.set_memory_growth(gpu, True)`: Configures TensorFlow to use GPU memory as needed instead of allocating all at once.

### **Matplotlib for Plotting**

In [ ]:
from matplotlib import pyplot as plt

* `matplotlib.pylot`: Library for creating static, animated, and interactive visualizations.

### **Import Custom Utilities and Modules**

In [ ]:
from utilities.configuration_classes import Config
from utilities.utilities import set_level, set_mpl_style, patch_mpl, set_global_rand_seed, dump_full_h5, load_full_h5
from utilities.utilities import info, warn, error, debug
from model.losses import configure_loss

from workflow.hls import hls

from workflow.plot_manager import HistPlotManager, RoCPlotManager

from workflow.train import train
from workflow.dataloader import DataLoader
from workflow.prepare_evaluation_object import prepare_evaluation_object
from workflow.trim_encoder import get_trimmed_vae, generate_trimmed_vae
from workflow.legacy_plot import legacy_plot

In that code block, `utilities` and `workflow` are custom modules created specifically for this AD project (they are not standard Python libraries. We can see them in `l1_anomaly_ae/dnn`).

* `Config`: Handles configuration settings.
* `set_level, set_mpl_style, patch_mpl, set_global_rand_seed, dump_full_h5, load_full_h5`: Various utility functions for logging, setting styles, seeding, and handling HDF5 files (set up the  log verbosity level, the matplotlib style for graphics and additional patches; set a global seed for random number generation; saves data on a HDF5 file, and load data from a HDF5 file).
* `info, warn, error, debug`: Logging functions.
* `configure_loss`: Configures the loss function for the model.
* `hls`: High-Level Synthesis related functionality.
* `HistPlotManager, RoCPlotManager`: Manages plotting of histograms and ROC curves.
* `train`: Function to train the model.
* `DataLoader`: Handles loading and preparing data.
* `prepare_evaluation_object`: Prepares the object for evaluation.
*`get_trimmed_vae, generate_trimmed_vae`: Functions to handle trimmed VAE models.
* `legacy_plot`: Handles legacy plotting.

### **Bit Extraction Function**

In [ ]:
def get_bits(s: str, pos: int, length: int = 4):
    if len(s) <= 2 and s.isdigit():
        i = int(s)
        assert 0 <= i <= 15
        s = bin(i)[2:].zfill(4)[::-1]
    if len(s) != length:
        return False
    for c in s:
        if c != '0' and c != '1':
            return False
    return s[pos] == '1'

* `get_bits`: Extracts bits from a binary string. Ensures the string is of correct length and composed of 0s and 1s. Used to check specific positions (and to determine whether this specific bit is on, i.e, 1).

### **Main Script Execution**

This code block checks if the script is running directly (not imported as a module), and if so, executes the following:

In [ ]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Train and test model')
    parser.add_argument('--config', type=str, default='dnn/config.yml', help='Configuration file with IO and training parameters setup')
    parser.add_argument('--run', type=str, default='all', help='what to run: train, eval, or all')
    parser.add_argument('--verbose-level', '-v', type=str, help='verbosity level, number or name', default='INFO')
    parser.add_argument('--force-cache-refresh', '-fr', action='store_true', help='force refresh of cache')
    args = parser.parse_args()
    set_level(args.verbose_level)

* `parser = argparse.ArgumentParser(...)`: Creates a ArgumentParser object with a description of the script.
* `parser.add_argument(...)`: Defined the arguments that the script can accept:
    * `--config`: Configuration archives route.
    * `--run`: Specifies the operation to do (train, eval, all).
    * `--verbose-level` (-v): log output verbosity level.
    * `--force-cache-refresh` (-fr): Force cache update if necessary.
* `args = parser.parse_args()`: Analize the command line arguments and save them in `args`.
* `set_level(args.verbose_level)`: Sets the logging verbosity level.

<span style="color: red;"> Key question: What is verbosity? Does this main script work by reading what you type in the terminal? </span>

### **Configuration and Output Directory Handling**

In [ ]:
    set_level(args.verbose_level)
    info(f"Using config file from {args.config}")
    config = Config.from_file(args.config)

    if os.path.isdir(config.output_path):
        if config.silent_overwrite:
            warn(f"Output path {config.output_path} already exists. Will overwriting silently.")
        else:
            input("Warning: output directory exists. Press Enter to continue...")
    else:
        os.mkdir(config.output_path)

    debug(f"Save config file to {config.output_path + '/config.json'}")
    config.dump(config.output_path + '/config.json')

The main purpose of this code block is to guarantee that the program has a secure place to storage its output files. If the output directory already exists, the program take the user configuration `silent_overwrite`. If it doesn't exist, the program creates the directory to ensure that can write the necessary files without errors.

* `Config.from_file`: Loads configuration from the specified YAML file.
* **Checks if output directory exists**: If it exists, warns the user (unless silent overwrite is enabled).
* Creates output directory if it doesn't exist.
* `config.dump`: Saves the configuration to a JSON file in the output directory.

### **Data Loading and Plot Settings**

In [ ]:
    dataloader = DataLoader(config.data)

    set_mpl_style()
    if config.save_json_for_plots:
        patch_mpl()

* `DataLoader(config.data)`: Initializes the data loader with configuration data. `dataloader` is an DataLoader object for handling data. `DataLoader` is a class that is responsible for loading (andre possibly preprocessing) the data necessary for training and evaluating the model. `config.data` is a configuration parameter that contains information about where the data is located, how it should be charged and another revelant configuration.
* `set_mpl_style()`: Sets the Matplotlib style (consistent and visually appealing).
* `patch_mpl()`: Patches Matplotlib if saving JSON for plots is enabled (applies necessary configuration to Matplotlib to support JSON functionality).

### **Global Seed and Cache Handling**

In [ ]:
    if config.global_seed is None:
        config.global_seed = np.random.randint(0, 1000000)

    info(f"Global random seed set to {config.global_seed}")
    set_global_rand_seed(config.global_seed)

    if args.force_cache_refresh:
        dataloader.load(force_refresh=True)

* Sets a random global seed if not provided. `if config.global_seed is None` checks if the global seed (`global_seed`) is defined. In the context of computational algorithms, the random number generators are usually pseudorandom number generators (PRNG), because the generated number sequences are determined by an initial seed. The seed acts as a starting point for the algorithm that generates the sequence.
* `set_global_rand_seed`: Sets the global random seed for reproducibility. `set_global_rand_seed(config.global_seed)` is responsible to apply the global seed to random number generators of used libraries as NumPy and TensorFlow. By **reproducibility** we mean the capablity to obtain the same results from an experiment each time it is repeated under the same conditions.
* Forces data cache refresh if specified.

### **Deterministic Mode and Workflow Steps**

In ML and data processing applications, reproducibility and determinism are important to ensure that experiments and results are consistent between run. This is relevant when using random operations or operations that may depend on multiple hardware factors.

In [ ]:
    if config.train.deterministic:
        tf.config.experimental.enable_op_determinism()
        warn("Deterministic mode is enabled. Performance may degrade.")

* Enables deterministic mode for reproducibility at the potential cost of performance. Sets TensorFlow for deterministic mode if it is specified in the configuration.
* `config.train.deterministic` indicates if the training should be realized on deterministic mode (it is a `boolean` flag established in the `.yml` file).
* `tf.config.experimental.enable_op_determinism()`: This function belongs to TensorFlow library and is used to turn on the deterministic mode for operations. With this, TensorFlow attempts to ensure that all operations are executed so that the results are the same each run (i.e, if the program is run several times under the same conditions, exactly the same results will be obtained), regardless of factors such as hardware or parallelization.
* `warn("Deterministic mode is enabled. Performance may degrade.")`: When you enable deterministic mode, ther may be a degradation in performance. This is because some hardware-dependent optimizations (such as parallelization on GPUs) may not support full determinisim, which can cause slower operations. 

### **Training Step**

In [ ]:
    if args.run == 'all' or 'train' in args.run or get_bits(args.run, 0):
        dataset = dataloader.get_dataset()
        scales, biases = dataset.norm_scale, dataset.norm_bias
        with h5.File(config.output_path + '/scales.h5', 'w') as f:
            f.create_dataset('norm_scale', data=scales)
            f.create_dataset('norm_bias', data=biases)
            
        configure_loss(dataset.norm_scale, dataset.norm_bias, config.data.constituents_mask)
        train(config, dataloader.get_dataset())
        generate_trimmed_vae(config)        

* Runs if `all`, `train`, or specific bit is set (that is, tells the `get_bits` function to verified if the first bit -position 0- in `args.run` is active).
* Loads (using `dataloader`) and normalizes (`norm_scale` and `norm_bias` are normalized data from `dataset`) dataset, save the scales and biases.
* Saves normalized scales and biases to an HDF5 file specified by `config.output_path`. That is useful for saving and loading these normalizations in future runs of the program.
* Configures loss function according to the normalized scales and the constituent mask defined in `config`.
* Trains the model using the configuration provided and the dataset obtained from `dataloader`.
* Generates a trimmed VAE model according `config`.

### **Evaluation Step**

In [ ]:
    if args.run == 'all' or 'eval' in args.run or get_bits(args.run, 1):
        dataset = dataloader.get_dataset()
        configure_loss(dataset.norm_scale, dataset.norm_bias, config.data.constituents_mask)
        prepare_evaluation_object(config, dataset)
        hls_model = hls(config, do_csim=True, dataset=dataset)

As in the previous code. This code block runs if `all`, `eval` or specific bit (the second bit, i.e, the -position 1- bit) is set; lodas dataset and configures loss function (using the same loss metrics as the training, ensuring consistency in the evaluation). However, we can see additionaly that:
* `prepare_evaluation_object`: Prepares the object to carry out the model evaluation (it may includes additional metrics, preparing specific data for evaluation, etc.).
* `hls`: Defined in `workflow/hls.py`, it is used to perform a HLS simulation. HLS is a process that transforms **High-Level** hardware descriptions into specific hardware. `do_csim=True` indicates that a behavioral simulation should be done. This part of the code prepares the object of evaluation and executes the workflow HLS.
* This evaluation step includes the exuction of a HLS workflow with **co-simulation** for validating and evaluating the model implementation in hardware. The **co-simulation** is a process which simulations from different domains (e.g. software and hardware) are combined to validate that both work correctly together. In HLS, co-simulation verifies that the HL model (usually wirtten in a language such as C/C++) behaves correctly and consistently when synsthesized in hardware. This ensure that HLS hardware design is correct and meets the HL model functionality expectatives.

### **Plotting Step**

Let's see the functions that are called to visualize the model's performanca and data (note that the graphics are also generated if the third bit -position 2- is active):

In [ ]:
    if args.run == 'all' or 'plot' in args.run or get_bits(args.run, 2):
        results = load_full_h5(config.output_path + '/results.h5')
        hist_plot_manager = HistPlotManager(config, results)
        roc_plot_manager = RoCPlotManager(config, results)
        input_plot_manager = InputPlotManager(config, results)
        roc_plot_manager.plot_eff_table()
        hist_plot_manager.plt_hls_dist(config.hls_metric)
        roc_plot_manager.plot_all_roc_curves(16)
        input_plot_manager.plot_all_input_hist()
        input_plot_manager.plot_all_input_correlation()

* `load_full_h5()`: Loads the evaluation results from an HDF5 file.
* `HistPlotManager();RoCPlotManager();InputPlotManager()`: Initializes plot managers for histograms, ROC curves, and input data.
* `plot_eff_table()`: Plots an efficiency table.
* `plt_hls_dist()`: Plots de HLS metric distribution.
* `plot_all_roc_curves(16)`: Plots all ROC curves.
* `plot_all_input_hist()`: Plots histograms of all input features.
* `plot_all_input_correlation()`: Plots correlation between all input features.

### **HLS (High-Level Synsthesis) Step**

Note that this code block will also run if the fourth bit -position 3- is active. This method could be used to active or desactive specific code parts based on a binary scheme. If `args.run` is a binary number represented by a string, this conditional will see the fourth bit.

In [ ]:
    if args.run == 'all' or 'hls' in args.run or get_bits(args.run, 3):
        hls_model = hls(config, do_csim=False, do_synth=True)

This code block is responsible for initializing the HLS based on the provided configurations. The conditional indicates that this process should be performed if the arguments provided to the program (`args.run`) allow it.

* Runs if `all`, `hls` or specific bit is set.
* `hls_model`: Generates the HLS model with synthesis (no co-simulation). Remember that HLS is a process that converts high-level descriptions of algorithms in low-level hardware (e.g. Verilog or VDHL).
* `do_csim=False`: Indicates that a behavioral simulation (C simulation) should not be performed. This type of simulation is typically used to validate the functionality of the algorithm before the synsthesis.
* `do_synth=True`: Indicates that the synthesis should be performed. Synthesis transforms the high level code into a hardware description, which is a key step for integrated circuits design.

By not performing the behavior simulation but doing the synthesis, the previous validation is skipped and we proceed directly to the generation of the hardware description. This part of the workflow is crucial for projects that involve designing hardware from high-level descriptions, allowing the creation of efficient implementations on FPGA or ASIC.